In [1]:
import pandas as pd
import numpy as np
import gc
import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.cross_validation import KFold, StratifiedKFold, cross_val_score
from datetime import datetime
#from catboost import CatBoostRegressor
from sklearn.preprocessing import StandardScaler,LabelEncoder, OneHotEncoder, minmax_scale, scale
from sklearn import tree
from sklearn import linear_model
from sklearn import svm
from sklearn import neighbors
from sklearn import ensemble
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
#from minepy import MINE
import tensorflow as tf
from numpy import nan as NA

E:\python-ana\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#drop相同列
def drop_col(data):
    data = data.fillna(data.mean())      
    for line in data.columns:
        if len(data[line].unique()) <= 3:
            data = data.drop([line], axis=1)    
    return data

In [3]:
#四分位数处理异常值
def quantile_dropout(data):  
    for c in data.columns:
        Q1 = data[c].quantile(q=0.25, interpolation='linear')
        Q3 = data[c].quantile(q=0.75, interpolation='linear')   
    
        min_v = Q1 - 1.5 * (Q3 - Q1)
        max_v = Q3 + 1.5 * (Q3 - Q1)
    
        data[c][(data[c] >= max_v) | (data[c] <= min_v)] = data[c].median()
           
    return data

In [4]:
#标准化
def scala(data):
 #   for col in data.columns:            
 #       data[col] = (data[col] - data[col].mean()) / data[col].std(ddof=0) 
 #   data = data.fillna(0)   
    data = scale(data)
    return data

In [5]:
def date_cols(data):
    for col in data:
        if data[col].min() > 1e13:
            data = data.drop([col], axis=1)   
    return data

In [6]:
def evalerror(y, y_pred):
    loss = np.sum(np.square(y - y_pred))
    n = len(y)
    return loss / n

In [7]:
train=pd.read_excel('D://train.xlsx')
testA=pd.read_excel('D://testA.xlsx')

In [8]:
#data_set = pd.read_csv('train/input_data.csv')
data_set=train.append(testA,ignore_index=True)

In [9]:
#y = pd.read_csv('train/y.csv')
y=data_set['Y']
data_set.drop(['Y'],axis=1,inplace=True)

In [10]:
data_set = pd.get_dummies(data_set, columns=['TOOL','TOOL (#1)','TOOL (#2)','TOOL_ID (#1)','TOOL_ID (#2)','TOOL_ID (#3)','Tool','Tool (#1)','Tool (#2)',
                               'Tool (#3)','tool','tool (#1)'])

In [11]:
data_set

,210X1,210X10,210X100,210X101,210X102,210X103,210X104,210X105,210X106,210X107,...,tool_4147,tool (#1)_P,tool (#1)_Q,tool (#1)_R,tool (#1)_S,tool (#1)_T,tool (#1)_U,tool (#1)_V,tool (#1)_W,tool (#1)_X
0,102.05,26.3,995.2,2000.0,1.60,0,5729.7,1.7,5732.1,0.7,...,0,0,0,0,0,0,0,0,1,0
1,100.95,28.2,1017.6,1999.8,1.65,0,5741.5,0.7,5736.9,3.1,...,0,0,0,0,0,0,0,0,0,1
2,98.56,26.6,1004.3,1999.8,1.64,0,5738.6,0.9,5747.2,0.6,...,0,0,0,0,0,0,0,0,1,0
3,100.35,25.2,1003.5,2000.2,1.66,0,5746.3,0.9,5743.9,3.5,...,0,0,0,0,0,0,0,0,0,1
4,100.25,27.3,1003.3,1999.8,1.66,0,5746.7,0.6,5741.6,3.1,...,0,0,0,0,0,0,0,0,1,0
5,100.55,25.3,1003.4,1999.9,1.66,0,5746.9,0.8,5742.4,2.4,...,0,0,0,0,0,0,0,0,1,0
6,102.20,26.5,1000.3,1999.9,1.60,0,5734.3,0.6,5734.7,0.9,...,0,0,0,0,0,0,0,0,0,1
7,102.25,27.1,1000.3,2000.2,1.60,0,5733.8,0.5,5734.4,0.7,...,0,0,0,0,0,0,0,0,1,0
8,102.25,25.9,1000.3,2000.1,1.60,0,5733.4,1.4,5734.3,1.0,...,0,0,0,0,0,0,0,0,0,1
9,100.85,27.7,1003.4,1999.8,1.66,0,5746.1,1.1,5742.2,3.1,...,0,0,0,0,0,0,0,1,0,0


In [12]:
data_set.drop(['ID', 'TOOL_ID'], axis=1, inplace=True)

In [13]:
#data_set = data_set.transpose().drop_duplicates().transpose()
data_set

,210X1,210X10,210X100,210X101,210X102,210X103,210X104,210X105,210X106,210X107,...,tool_4147,tool (#1)_P,tool (#1)_Q,tool (#1)_R,tool (#1)_S,tool (#1)_T,tool (#1)_U,tool (#1)_V,tool (#1)_W,tool (#1)_X
0,102.05,26.3,995.2,2000.0,1.60,0,5729.7,1.7,5732.1,0.7,...,0,0,0,0,0,0,0,0,1,0
1,100.95,28.2,1017.6,1999.8,1.65,0,5741.5,0.7,5736.9,3.1,...,0,0,0,0,0,0,0,0,0,1
2,98.56,26.6,1004.3,1999.8,1.64,0,5738.6,0.9,5747.2,0.6,...,0,0,0,0,0,0,0,0,1,0
3,100.35,25.2,1003.5,2000.2,1.66,0,5746.3,0.9,5743.9,3.5,...,0,0,0,0,0,0,0,0,0,1
4,100.25,27.3,1003.3,1999.8,1.66,0,5746.7,0.6,5741.6,3.1,...,0,0,0,0,0,0,0,0,1,0
5,100.55,25.3,1003.4,1999.9,1.66,0,5746.9,0.8,5742.4,2.4,...,0,0,0,0,0,0,0,0,1,0
6,102.20,26.5,1000.3,1999.9,1.60,0,5734.3,0.6,5734.7,0.9,...,0,0,0,0,0,0,0,0,0,1
7,102.25,27.1,1000.3,2000.2,1.60,0,5733.8,0.5,5734.4,0.7,...,0,0,0,0,0,0,0,0,1,0
8,102.25,25.9,1000.3,2000.1,1.60,0,5733.4,1.4,5734.3,1.0,...,0,0,0,0,0,0,0,0,0,1
9,100.85,27.7,1003.4,1999.8,1.66,0,5746.1,1.1,5742.2,3.1,...,0,0,0,0,0,0,0,1,0,0


In [14]:
data_X = data_set.ix[:499, :]
data_X_test = data_set.ix[500:, :]
data_y = y.ix[:499]

E:\python-ana\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
E:\python-ana\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [15]:
np.where(np.isnan(data_X))

(array([  0,   0,   0, ..., 499, 499, 499], dtype=int64),
 array([ 274,  275, 1618, ..., 3879, 3881, 3883], dtype=int64))

In [16]:
data_X.fillna(data_X.mean(),inplace=True)
data_X_test.fillna(data_X_test.mean(),inplace=True)

E:\python-ana\lib\site-packages\pandas\core\generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,210X1,210X10,210X100,210X101,210X102,210X103,210X104,210X105,210X106,210X107,...,tool_4147,tool (#1)_P,tool (#1)_Q,tool (#1)_R,tool (#1)_S,tool (#1)_T,tool (#1)_U,tool (#1)_V,tool (#1)_W,tool (#1)_X
0,102.05,26.3,995.2,2000.0,1.60,0,5729.7,1.7,5732.1,0.7,...,0,0,0,0,0,0,0,0,1,0
1,100.95,28.2,1017.6,1999.8,1.65,0,5741.5,0.7,5736.9,3.1,...,0,0,0,0,0,0,0,0,0,1
2,98.56,26.6,1004.3,1999.8,1.64,0,5738.6,0.9,5747.2,0.6,...,0,0,0,0,0,0,0,0,1,0
3,100.35,25.2,1003.5,2000.2,1.66,0,5746.3,0.9,5743.9,3.5,...,0,0,0,0,0,0,0,0,0,1
4,100.25,27.3,1003.3,1999.8,1.66,0,5746.7,0.6,5741.6,3.1,...,0,0,0,0,0,0,0,0,1,0
5,100.55,25.3,1003.4,1999.9,1.66,0,5746.9,0.8,5742.4,2.4,...,0,0,0,0,0,0,0,0,1,0
6,102.20,26.5,1000.3,1999.9,1.60,0,5734.3,0.6,5734.7,0.9,...,0,0,0,0,0,0,0,0,0,1
7,102.25,27.1,1000.3,2000.2,1.60,0,5733.8,0.5,5734.4,0.7,...,0,0,0,0,0,0,0,0,1,0
8,102.25,25.9,1000.3,2000.1,1.60,0,5733.4,1.4,5734.3,1.0,...,0,0,0,0,0,0,0,0,0,1
9,100.85,27.7,1003.4,1999.8,1.66,0,5746.1,1.1,5742.2,3.1,...,0,0,0,0,0,0,0,1,0,0


In [17]:
data_X.dropna(how='any',axis=1,inplace=True)
data_X_test.dropna(how='any',axis=1,inplace=True)

E:\python-ana\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
def norm(data_X):
    for j in [col for col in data_X]:
        t_max=data_X[j].max()
        t_min=data_X[j].min()
        down = 1 if (t_max-t_min == 0) else (t_max-t_min)
        data_X[j]=(data_X[j]-t_min)/down

In [19]:
norm(data_X)
norm(data_X_test)
data_X

E:\python-ana\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,210X1,210X10,210X100,210X101,210X102,210X103,210X104,210X105,210X106,210X107,...,tool_4147,tool (#1)_P,tool (#1)_Q,tool (#1)_R,tool (#1)_S,tool (#1)_T,tool (#1)_U,tool (#1)_V,tool (#1)_W,tool (#1)_X
0,0.992704,0.894558,0.963967,0.99960,0.958084,0.0,0.995275,0.809524,0.995571,0.134615,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.982004,0.959184,0.985664,0.99950,0.988024,0.0,0.997325,0.333333,0.996405,0.596154,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.958755,0.904762,0.972782,0.99950,0.982036,0.0,0.996821,0.428571,0.998194,0.115385,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.976167,0.857143,0.972007,0.99970,0.994012,0.0,0.998159,0.428571,0.997621,0.673077,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.975195,0.928571,0.971813,0.99950,0.994012,0.0,0.998228,0.285714,0.997221,0.596154,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5,0.978113,0.860544,0.971910,0.99955,0.994012,0.0,0.998263,0.380952,0.997360,0.461538,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
6,0.994163,0.901361,0.968907,0.99955,0.958084,0.0,0.996074,0.285714,0.996023,0.173077,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7,0.994650,0.921769,0.968907,0.99970,0.958084,0.0,0.995987,0.238095,0.995971,0.134615,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,0.994650,0.880952,0.968907,0.99965,0.958084,0.0,0.995918,0.666667,0.995953,0.192308,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9,0.981031,0.942177,0.971910,0.99950,0.994012,0.0,0.998124,0.523810,0.997325,0.596154,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
f_num=8004
LEARNING_RATE_BASE = 0.005
LEARNING_RATE_DECAY = 0.99
training_steps=500
HIDDEN_SIZE=2000
OUTPUT_SIZE=1

In [ ]:
with tf.variable_scope("dnn",reuse=tf.AUTO_REUSE):
    X = tf.placeholder("float",[None,f_num,])
    Y = tf.placeholder("float", [None,OUTPUT_SIZE])
    global_step = tf.Variable(0, trainable=False)
    
    #lay_1 = tf.Variable(tf.random_normal([f_num, HIDDEN_SIZE], mean=0.0, stddev=1.0))
    #lay_2 = tf.Variable(tf.random_normal([HIDDEN_SIZE,OUTPUT_SIZE],mean=0.0,stddev=1.0))
    lay_1 = tf.Variable(tf.zeros([f_num, HIDDEN_SIZE]))
    lay_2 = tf.Variable(tf.zeros([HIDDEN_SIZE,OUTPUT_SIZE]))
    b_1 = tf.Variable(tf.zeros([HIDDEN_SIZE]),dtype="float32",name="bias_1")
    b_2 = tf.Variable(tf.zeros([OUTPUT_SIZE]),dtype="float32",name="bias_2")
    
    lay_out1 = tf.nn.sigmoid(tf.matmul(tf.reshape(X,[-1,f_num]), lay_1) + b_1)
    
    y_=tf.add(tf.matmul(tf.reshape(lay_out1,[-1,HIDDEN_SIZE]),lay_2),b_2)
    di=y_-Y
    loss_op = tf.reduce_mean(tf.square(y_ - Y))
    loss=tf.sqrt(loss_op)
    learning_rate = tf.train.exponential_decay(
            LEARNING_RATE_BASE,
            global_step,
            200, LEARNING_RATE_DECAY,
            staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(loss_op,global_step=global_step)
    
    # Initialize the variables (i.e. assign their default value)
    init = tf.global_variables_initializer()

In [ ]:
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    #test_X=[]
    #test_y=[]
    #for i in range(len(date_list)):
    #test_X=(np.array(date_list[0],dtype=np.float32)[0:5])
    #test_y=(np.array([ [sale_amt_list[0].iloc[i]] for i in range(5)]))
    for step in range(training_steps):
        train_X=np.array([data_X.ix[step]],dtype=np.float32)
        train_y=np.array([[data_y.ix[step]]],dtype=np.float32)
        test_X=np.array(data_X_test,dtype=np.float32)
        sess.run(train_op, feed_dict={X: train_X, Y: train_y})
            
        if step % 10 == 0:
            pass
                #print(sess.run(loss,feed_dict={X:np.array(date_list[0].iloc[5],dtype=np.float32), Y: np.array(sale_list[0].iloc[-1],dtype=np.float32)}))
            print(sess.run([loss],feed_dict={X:train_X, Y: train_y}))
    pre=sess.run(y_,feed_dict={X:data_X})
    result=sees.run(y_,feed_dict={X:test_X})

In [ ]:
pre

In [ ]:
data_y

In [ ]:
t=[]
for  i in pre:
    t.append(i[0])
tt=np.array(data_y)
s=0
for i in range(len(t)):
    s=s+pow(t[i]-tt[i],2)
s/len(t)